> # LECTURE - 01
>
> ### INTRODUCTION TO LINEAR REGRESSION: STOCK PREDICTION<br><br>


<pre>
- The Idea of the regression is to take continuous data and 
  figure out the best fit line for that data.

- With Linear Regression, we try to model our data with a straight line.

- Equation of the stringht line is: y = mx + c
    if we have x, we can figure out y given that we have m and c

- Basically the whole point of the regresion is to find out what m and c is.
</pre>


<pre>
- With ML, especially with Supervised ML, everything boils down to Features(are Attributes, in the below case it is continous data)
  and Lebels.
</pre>


In [1]:
import quandl
import math
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

In [2]:
df = quandl.get('WIKI/GOOGL')
print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

<li>In Machine Learning, We can have all the features we want but we need to have only some meaningful features, features that actually have to do something with our data.</li>


In [3]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
print(df.head)

<bound method NDFrame.head of               Adj. Open    Adj. High     Adj. Low   Adj. Close  Adj. Volume
Date                                                                       
2004-08-19    50.159839    52.191109    48.128568    50.322842   44659000.0
2004-08-20    50.661387    54.708881    50.405597    54.322689   22834300.0
2004-08-23    55.551482    56.915693    54.693835    54.869377   18256100.0
2004-08-24    55.792225    55.972783    51.945350    52.597363   15247300.0
2004-08-25    52.542193    54.167209    52.100830    53.164113    9188600.0
...                 ...          ...          ...          ...          ...
2018-03-21  1092.570000  1108.700000  1087.210000  1094.000000    1990515.0
2018-03-22  1080.010000  1083.920000  1049.640000  1053.150000    3418154.0
2018-03-23  1051.370000  1066.780000  1024.870000  1026.550000    2413517.0
2018-03-26  1050.600000  1059.270000  1010.580000  1054.090000    3272409.0
2018-03-27  1063.900000  1064.540000   997.620000  1006.94

In [7]:
# Percentage change in High to Low stock price:
# This is almost percent votatility of the day
# Therefore, we are gonna to define a new column: "H2L_PerCng"


df['H2L_PerCng'] = (df['Adj. High'] - df['Adj. Close']) / \
    df['Adj. Close'] * 100.0

In [9]:
# (Features)
# Daily Percentage change or the daily move in Close to Open stock price:
# Normally the Percent Change is => ((New - Old) / (Old)) * 100

df['Per_Cng'] = (df["Adj. Close"] - df['Adj. Open']) / df['Adj. Open'] * 100.0

In [11]:
# (Features)
# Defining the new dataframe, we want to work with or we do really care about:

df = df[['Adj. Close', 'H2L_PerCng', 'Per_Cng', 'Adj. Volume']]

print(df.head())

            Adj. Close  H2L_PerCng   Per_Cng  Adj. Volume
Date                                                     
2004-08-19   50.322842    3.712563  0.324968   44659000.0
2004-08-20   54.322689    0.710922  7.227007   22834300.0
2004-08-23   54.869377    3.729433 -1.227880   18256100.0
2004-08-24   52.597363    6.417469 -5.726357   15247300.0
2004-08-25   53.164113    1.886792  1.183658    9188600.0


<br><br>

> # LECTURE - 02
>
> ### Features and Labels:
>
> Usually we define features by capital X and labels by small y.<br><br>


In [31]:
# It is just a variable created to work with the stock prices:
forecast_col = 'Adj. Close'


# Just in case there is not missing data
# In case of missing data and we can't get rid of it, so we fill that column with some specific values:
df.fillna(-9999, inplace=True)
# We did this because, with ML we can't work with NaN data,
# therefore we actually have to replace the NaN with something, or
# we can get rid of the entire column.


# Actually we are trying to predict the 10% of the dataframe to predict the forecast out:
forecast_out = int(math.ceil(0.01 * len(df)))
print(f'forecast_out is: {forecast_out} \n\n')

# Creating label:
# Since we have forecast_out, so we can create labels
# Here we are using forecast_out as minus that's why we made it as int data type
# So, basically we are shifting the columns negativelly upwards.
df['label'] = df[forecast_col].shift(-forecast_out)
# This way, each row, the label column for each row will be adjusted close price 10 days into the future.
# So, that's our label.


# So, this will just print first five rows of the dataframes
print(f'dataframe head is:\n {df.head()}\n\n')

df.dropna(inplace=True)

print(f'dataframe tail is:\n {df.tail()}')

forecast_out is: 31 


dataframe head is:
             Adj. Close  H2L_PerCng   Per_Cng  Adj. Volume      label
Date                                                                
2004-08-19   50.322842    3.712563  0.324968   44659000.0  67.739104
2004-08-20   54.322689    0.710922  7.227007   22834300.0  69.399229
2004-08-23   54.869377    3.729433 -1.227880   18256100.0  68.752232
2004-08-24   52.597363    6.417469 -5.726357   15247300.0  69.639972
2004-08-25   53.164113    1.886792  1.183658    9188600.0  69.078238


dataframe tail is:
             Adj. Close  H2L_PerCng   Per_Cng  Adj. Volume   label
Date                                                             
2016-08-24      793.60    0.612399 -0.409106    1284437.0  800.71
2016-08-25      791.30    0.432200 -0.088384    1202680.0  814.17
2016-08-26      793.22    0.779103  0.092115    1248881.0  809.57
2016-08-29      795.82    0.339273  0.349284     773698.0  811.77
2016-08-30      791.92    0.767754 -0.121078    1167413.

<br><br>

> # LECTURE - 03
>
> ### Training and Testing: <br><br>


<li>Scaling our data aspect is usually done on the features.</li>
<li>The goal is often to get our features to be somewhere between -1 and +1</li>
<li>This can actaully just help with accuracy as well as just processing speed or how long it might take to actually do the calculations.</li>
<br>

<li>Here, we are gonna to use cross-validation to create our training and testing samples. It's a good way to split up our data, suffle it up.</li>


In [20]:
# so, our features are basically everything, except for the label column.
# We can do this beacause df.drop() return new dataframes and
# it can be converted to numpy array and here it is being saved to the value of the X.
X = np.array(df.drop(['label'], axis=1))


# Creating the labels:
# Now the value of y is our labels :
y = np.array(df['label'])


# Now, we are going to scale X


X = preprocessing.scale(X)


# redefining X, this includes all the point because we have shifted
# So, we are actually making sure that we only have X's where we have values for 'y'
# X = X[:-forecast_out + 1]
# We actually don't need to do it again, as we have already droped it above.


print(len(X), len(y))

3321 3321


In [36]:
# Since, we got our X's and 'y'
# therefore we are now ready for creating our training and testing sets.
# Creating training and testing set of data(hopefully here we are using 20% of total data)
# X_test and y_test we used to fit our classifier.
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2)
# We use, X_train and y_train, to fit our classifiers.

# Stuff that is actually goinig to predict again:
X_lately = X[-forecast_out:]


# Now we need to define the classifier
clf = LinearRegression()


# Now to fit or train our classifier, we just fit our classifier
# fit is synonymous with train
clf.fit(X_train, y_train)


# Now we need to see out what the accuracy is
# score is synonymous with test
accuracy = clf.score(X_test, y_test)


print(f'Linear Regression Model Accuracy is: {accuracy}')

Linear Regression Model Accuracy is: 0.9775392066724172


In [38]:
# Switching The Algorithm From Linear Regression to SVM
# Let's now we try to use a different algorithm: SVM, just for testing
# Default value of kernel in the SVR() is 'linear', we can also use 'poly' as well:
# clf = svm.SVR(kernel = 'poly')

clf = svm.SVR()

clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test)

print(f'SVM Model Accuracy is: {accuracy}')

SVM Model Accuracy is: 0.8055514658688122


<br><br>

> # LECTURE - 04
>
> ### Forecasting and Predicting: <br><br>


The parameter 'n_jobs' means that how many jobs or how many threads we are willing to run at any given time.

And the default for regression is 1.

'clf = LinearRegression()' => this is running in linealy. <br>
'clf = LinearRegression(n_jobs = 10)' => this will run atleast 10 jobs at a time, parallelly. <br>

'clf = LinearRegression(n_jobs = -1)' => this will run as many jobs as possible by our processor.
